## ROC extraction

on commence par regarder les documents ou l on observe le mot center


In [39]:
# SYSTEM
from os import listdir
import sys
sys.path.insert(0,'/Users/greghor/Documents/taf/UGent/python_prog/py_commonFunc/')
from PyCommonFun import *

#NLTK
import nltk, re, pprint
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import words
from nltk.tokenize import RegexpTokenizer
from nltk.metrics import *
from nltk.text import Text

#GENERAL
import numpy as np
import pandas as pd
import pickle
import string

In [126]:
# fonctions used in the notebook

def getContext(s,pattern,context=list(),width=100):
    
    
    loc=s.find(pattern)
    
    if loc == -1:
        return context
    elif loc-width < 0:
        context.append(s[:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    elif loc + width > len(s):
        context.append(s[loc-width:])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    else:
        context.append(s[loc-width:loc+width])
        return getContext(s[loc+len(pattern):],pattern,context,width)
    
    
def useSolutionTypo(s):
    s=s.lower()
    if s=="sao paulo":
        s="sao-paulo"
    elif s=='target 1' or s =='target 2' or s == 'target' or s=='target1':
        s='target holidays'
    return s



def document_from_hmtl(doc_path):
    with open(doc_path) as html_file:
        full_text = html_file.read()
        body = re.findall(r'<body[^>]*?>(.*?)</body>', full_text)
        cleantext = re.sub('<\/?span[^>]*>', ' ', body[0])
        clean_text = re.sub('<.*?>', ' ', cleantext)
        clean_text = clean_text.lower()
        clean_text = re.sub('0.01', '1', clean_text)
        clean_text = re.sub(',', '', clean_text)
        clean_text = re.sub(r'[^a-z0-9]', ' ', clean_text)
        clean_text = re.sub('\s+', ' ', clean_text)
        return clean_text


In [132]:
htmlFiles=listdir('../../data_0/html/')
print PyFind(htmlFiles, lambda x: '0900045c83c2ece7' in x )
print htmlFiles[1178]

[1178]
0900045c83c2ece7_IXS1339589837_F_PC_N_M_ocr.html


In [128]:
text=document_from_hmtl(dataPath + '/html/0900045c83c2ece7_IXS1339589837_F_PC_N_M_ocr.html')
print text

 no prospectus is required in accordance with directive 2003 71 ec for the issue of notes described below pricing supplement dated 28 january 2016 royal bank of canada a canadian chartered bank issue of hkd 6000000 discounted fund linked redemption notes linked to hang seng h share index etf due february 2016 under the programme for the issuance of securities neither the issuer nor any dealer has authorised nor do they authorise the making of any offer of notes in any other circumstances part a contractual terms any person making or intending to make an offer of the notes may only do so in circumstances in which no obligation arises for the issuer or any dealer to publish a prospectus pursuant to article 3 of the prospectus directive or to supplement a prospectus pursuant to article 16 of the prospectus directive in each case in relation to such offer this document constitutes the pricing supplement for the notes described herein this document must be read in conjunction with the struc

In [40]:
# data path
curDir=%pwd
if curDir=='/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/notebook':
    dataPath='/Users/greghor/Documents/taf/dataScience/Euroclear_hackathon/dataset/data_0_labels' 
                   
else:
    dataPath='../../data_0'

In [41]:
# get the solution
labels=pd.read_csv(dataPath + '/labels.csv')
ROCTmp=pd.read_csv('ROC_fullList.csv',delimiter=";")
cities_groundTruth=pd.read_csv(dataPath + '/cities.csv')
ROC_list=['target 1','target 2','target'] # add target 1 and target 2
# lowerise ROC and remove ponctuation
for roc in ROCTmp['CIT_NM'].values:
    roc=str(roc).lower().translate(None, string.punctuation)
    ROC_list.append(roc.rstrip())

In [44]:
# add roc from operationnal curency
roc_fromCurrency=pd.read_csv('currency_ROC_fullList.csv',delimiter=";")
# merge with labels on operationnal currency
roc_fromCurrency=pd.merge(left=roc_fromCurrency,right=labels,left_on='Currency Code', right_on='OpCurrency')
# keep only currency code and default ROC columns
col=["isin","OpCurrency","Default ROC",]
roc_fromCurrency=roc_fromCurrency[col]
roc_fromCurrency.rename(columns={'Default ROC':'currency_ROC'},inplace=True)
roc_fromCurrency.head()

isin OpCurrency      currency_ROC
0  XS1293836406        AUD  Sydney|Melbourne
1  XS1293840770        AUD  Sydney|Melbourne
2  XS1293141849        AUD  Sydney|Melbourne
3  XS1340008645        AUD  Sydney|Melbourne
4  XS1267292354        AUD  Sydney|Melbourne

In [115]:
text=''
f='0900045c81d0d57e'
fileOpen=open(dataPath + '/html/0900045c83c2ece7_IXS1339589837_F_PC_N_M_ocr')
text=text+fileOpen.read()
text=unicode(text, 'ascii', 'ignore')    
print type(text)

<type 'unicode'>


In [123]:
text=document_from_hmtl(dataPath + '/html/0900045c83c2ece7_IXS1339589837_F_PC_N_M_ocr.html')

AttributeError: 'file' object has no attribute 'decode'

In [101]:
gf=0


context_bdc=dict()


for isin in np.unique(labels['isin'].values):
    
    fid=labels['fileId'].loc[labels['isin']==isin]
    
    text=''
    for f in fid:
    
    '''
    fileOpen=open(dataPath + '/txt/' + f + '_ocr.txt')
        text=text+fileOpen.read()
        text=unicode(text, 'ascii', 'ignore')
        text=str(text).lower()
        # remove some annoying punctutations
        text=text.translate(None, string.punctuation)
        text=text.translate(None,'\r\n\x0c')
    '''
        
    
    
    context_bdc[isin]=dict()
    sentence=sent_tokenize(text)
    
    pattern=["Business Day Centre","Financial Centre","Business Centre","Currency Business Day",'Business Day']
    
    for pat in pattern:
        context_bdc[isin][pat]=getContext(text,pat.lower(),[],250)
        # il faut impérativement passer, un troisieme argument vide pour que le code 
        # fonctionne mais je ne comprends pas bien pourquoi
            
            
            
    gf+=1
       


In [102]:
# try to extract the ROC 

extractedROC_dict=dict()

for isin in context_bdc.keys():
    
    extractedROC_dict[isin]=dict()
    currencyROCTmp=roc_fromCurrency['currency_ROC'].loc[roc_fromCurrency['isin']==isin].values[0].lower()
    extractedROC_dict[isin]['currencyROC']=useSolutionTypo(currencyROCTmp)
    
    for pat in context_bdc[isin].keys():
    
        extractedROC_dict[isin][pat]=''
        
        for roc in ROC_list:
            
            if roc in ''.join(context_bdc[isin][pat]):
                
                # not clean but makes things much easier for 
                roc=useSolutionTypo(roc)
            
                    
                
                if len(extractedROC_dict[isin][pat])==0:
                    extractedROC_dict[isin][pat]=roc
                else:
                    extractedROC_dict[isin][pat]=extractedROC_dict[isin][pat]+'|'+roc
                    
                    
               # if currencyROCTmp  not in currencyROCTmp:
               #     extractedROC_dict[isin][pat]=extractedROC_dict[isin][pat]+'|'+ currencyROCTmp
                
                
                
                 

In [103]:
useSolutionTypo('target 1')

'target holidays'

In [104]:
extractedROC_df=pd.DataFrame()
extractedROC_df['isin']=[]
for pat in extractedROC_dict['XS1289653195'].keys():
    extractedROC_df[pat]=[]




In [105]:
for isin in extractedROC_dict.keys():
    newRow=[isin]
    for pat in extractedROC_dict[isin].keys():
        newRow.append(extractedROC_dict[isin][pat].upper())
    extractedROC_df.loc[len(extractedROC_df)]=newRow

In [106]:
print isin
print pat
print extractedROC_dict[isin][pat]

XS1349021326
Business Day Centre



In [107]:
extractedROC_df

isin                                      Business Day  \
0     XS1240015799                                          NEW YORK   
1     XS1293137904                                   TARGET HOLIDAYS   
2     XS1313796846                                                     
3     XS1051380837                                                     
4     XS1313800895                                             TOKYO   
5     XS1267246327                          TARGET HOLIDAYS|NEW YORK   
6     XS1275964507                   LONDON|NEW YORK|SAO-PAULO|TOKYO   
7     XS1329979261                                                     
8     XS1326567895                                                     
9     XS1253932765  BRUSSELS|EUROCLEAR|LUXEMBOURG|NEW YORK|SINGAPORE   
10    XS1267246244                                                     
11    XS1293817877                                         HONG KONG   
12    XS1051370440                                      LONDON|TOKYO   
13    XS1267221841                                                     
14    XS1266266813                                         EUROCLEAR   
15    XS1313804459                                                     
16    XS1332626016                                                     
17    XS1338871996                                      LONDON|TOKYO   
18    XS1313802677                                                     
19    XS1313800119                                                     
20    XS1293807720                           HONG KONG|LIMA|NEW YORK   
21    XS1051345681                                      LONDON|TOKYO   
22    XS1222904416                                                     
23    XS1190866803                                                     
24    XS1281373271                      EUROCLEAR|HONG KONG|NEW YORK   
25    XS1267238498                                            LONDON   
26    XS1254344952                                         HONG KONG   
27    XS1293112667                                   TARGET HOLIDAYS   
28    XS1331674041                                                     
29    XS1251673817                                                     
...            ...                                               ...   
1170  XS1338471466                                   TARGET HOLIDAYS   
1171  XS1324891065                                          NEW YORK   
1172  XS1212135088                          TARGET HOLIDAYS|NEW YORK   
1173  XS1324219192                                      LONDON|TOKYO   
1174  XS1281376027                      EUROCLEAR|HONG KONG|NEW YORK   
1175  XS1313396662                                                     
1176  XS1336597577                                            ZURICH   
1177  XS1324931036                                      LONDON|TOKYO   
1178  XS1326551030                                                     
1179  XS1254287979                                         HONG KONG   
1180  XS1326560536                                                     
1181  XS1199690345                                         EUROCLEAR   
1182  XS1338163303                                      LONDON|TOKYO   
1183  XS1293833056                                         HONG KONG   
1184  XS1190845807                                                     
1185  XS1051369434                                                     
1186  XS1266253084                                                     
1187  XS1051364559                                                     
1188  XS1339629567                                                     
1189  XS1176700158                             LONDON|NEW YORK|TOKYO   
1190  XS1293110612                                            ZURICH   
1191  XS1293820749                                         HONG KONG   
1192  XS1301496003                                         HONG KONG   
1193  XS1210455322                                    

In [108]:
# merge with citiesGreg to check how many results are OK

citiesGreg=pd.read_csv('citiesGreg.csv')
joinedCities=pd.merge(citiesGreg,extractedROC_df,on='isin')

In [109]:
# check the compliance between ground truth and my solution
for pat in extractedROC_dict['XS1289653195'].keys():
    indCheck=joinedCities["citiesName"] == joinedCities[pat]
    joinedCities["check_" + pat ]=indCheck

# reorder the columns for clarity
col=['fileId',
 'isin',
 'citiesName',
 'Financial Centre','check_Financial Centre',
 'Business Day Centre', 'check_Business Day Centre',
 'Currency Business Day', 'check_Currency Business Day',
 'Business Day', 'check_Business Day',
  'currencyROC','check_currencyROC']


In [110]:
check_extractedCities=joinedCities[col]
check_extractedCities.to_csv('check_extractedCities.csv')

In [111]:
check_extractedCities

fileId          isin                       citiesName  \
0     0900045c83a87e4a  XS1240015799                         NEW YORK   
1     0900045c83c101a8  XS1293137904                  TARGET HOLIDAYS   
2     0900045c83ac888d  XS1313796846                     LONDON|TOKYO   
3     0900045c83b42aaf  XS1313802677                   NEW YORK|TOKYO   
4     0900045c83b36448  XS1313800895                     LONDON|TOKYO   
5     0900045c83a715b0  XS1267246327         NEW YORK|TARGET HOLIDAYS   
6     0900045c8365c27b  XS1275964507  LONDON|NEW YORK|SAO-PAULO|TOKYO   
7     0900045c83aab72e  XS1329979261                            TOKYO   
8     0900045c83af0dc8  XS1312160507                         NEW YORK   
9     0900045c833b5bd8  XS1253932765               NEW YORK|SINGAPORE   
10    0900045c83b0c28d  XS1267246244         NEW YORK|TARGET HOLIDAYS   
11    0900045c83b9478c  XS1293817877                        HONG KONG   
12    0900045c83ba04ae  XS1051370440                     LONDON|TOKYO   
13    0900045c83a8e987  XS1267221841         NEW YORK|TARGET HOLIDAYS   
14    0900045c83bbdb52  XS1266266813           LONDON|TARGET HOLIDAYS   
15    0900045c83b527cb  XS1313804459                   NEW YORK|TOKYO   
16    0900045c83b140f2  XS1332626016                  LONDON|NEW YORK   
17    0900045c83b9fc0b  XS1338871996                     LONDON|TOKYO   
18    0900045c83c6ccdc  XS1051380837                     LONDON|TOKYO   
19    0900045c83b6355c  XS1313800119            LONDON|NEW YORK|TOKYO   
20    0900045c83b6155d  XS1293807720               HONG KONG|NEW YORK   
21    0900045c83af1007  XS1051345681                     LONDON|TOKYO   
22    0900045c83afb3ab  XS1326548325                        HONG KONG   
23    0900045c83c2ecd4  XS1339630227                         NEW YORK   
24    0900045c83a54cdb  XS1267238498                     LONDON|TOKYO   
25    0900045c83b25fa7  XS1254344952                        HONG KONG   
26    0900045c83b0569e  XS1293112667                  TARGET HOLIDAYS   
27    0900045c83b294a0  XS1293113392                           MEXICO   
28    0900045c83a80205  XS1331674041                  LONDON|NEW YORK   
29    0900045c833768ef  XS1251673817                         NEW YORK   
...                ...           ...                              ...   
1169  0900045c83439fa1  XS1212135088         NEW YORK|TARGET HOLIDAYS   
1170  0900045c83b25f30  XS1289331826                        HONG KONG   
1171  0900045c83a5d28a  XS1281376027                         NEW YORK   
1172  0900045c83c2029d  XS1313396662            LONDON|NEW YORK|TOKYO   
1173  0900045c83bf47e5  XS1336597577                           ZURICH   
1174  0900045c83a61e00  XS1324931036                     LONDON|TOKYO   
1175  0900045c83b0ff86  XS1293836232                        HONG KONG   
1176  0900045c83a65762  XS1326560536                        HONG KONG   
1177  0900045c83070368  XS1199690345                           LONDON   
1178  0900045c82fccfd4  XS1183240487           LONDON|TARGET HOLIDAYS   
1179  0900045c83b89e81  XS1340833521                     LONDON|TOKYO   
1180  0900045c83b26b3f  XS1293833056                        HONG KONG   
1181  0900045c8282b092  XS1078109946                         NEW YORK   
1182  0900045c8389f375  XS1310504615                         NEW YORK   
1183  0900045c838dc9cf  XS1266253084                  LONDON|NEW YORK   
1184  0900045c83003bb6  XS1199927614  LONDON|NEW YORK|SAO-PAULO|TOKYO   
1185  0900045c83c3fb7b  XS1339629567                        SINGAPORE   
1186  0900045c8334529d  XS1229484396                  LONDON|NEW YORK   
1187  0900045c83ac9cf0  XS1293108806                         NEW YORK   
1188  0900045c83b05686  XS1293110612                           ZURICH   
1189  0900045c835ab195  XS1274143350                  LONDON|NEW YORK   
1190  0900045c83c23ed4  XS1293139199                  TARGET HOLIDAYS   
1191  0900045c83a5d7d1  XS1301496003                        HONG KONG   
1

In [ ]:
" election is given to the Noteholders in".index('to the')

In [ ]:
tok=word_tokenize(str(context_bdc["0900045c83a966e3"]))
toto=nltk.Text(tok)

In [ ]:
np.where(text=='Issuer')

In [ ]:
toto=re.search('Business Day Centre',text)

In [ ]:
toto.group()

In [ ]:
toto=text.common_contexts("Centre")

In [ ]:
text.findall("<.*><Day Centre><.*>")

In [ ]:
moby = Text(nltk.corpus.gutenberg.words('melville-moby_dick.txt'))

In [ ]:
moby

In [ ]:
moby.common_contexts('whale')

In [ ]:
toto

In [ ]:
toto

In [ ]:
text.concordance('prospectus')


In [ ]:
type(text)

In [ ]:
f

In [ ]:
f

In [ ]:
f

In [ ]:
for f in labels[0

In [ ]:
labels.fileId.values[0]

In [ ]:
for f in labels.fileId:
    print f